<a href="https://colab.research.google.com/github/kingketan9/BigDataLabs/blob/main/LabEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=fb79fcb947f2db43e3adc536e78c5122c19fec439f66d62e7fcbcd30dd0bc06c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType

spark = SparkSession.builder.appName("CreateDataFrame").getOrCreate()

schema = StructType([
    StructField("Customer_id", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Product_1", IntegerType(), True),
    StructField("Product_2", IntegerType(), True),
    StructField("Product_3", IntegerType(), True)
])

data = [
    (1, 'Alice', 2000, None, 3000),
    (3, 'Carol', None, 4000, None),
    (5, 'John', 1000, None, 3000),
    (6, 'Michael', None, 2000, 3000),
]

df = spark.createDataFrame(data, schema=schema)

df.show()

+-----------+-------+---------+---------+---------+
|Customer_id|   Name|Product_1|Product_2|Product_3|
+-----------+-------+---------+---------+---------+
|          1|  Alice|     2000|     null|     3000|
|          3|  Carol|     null|     4000|     null|
|          5|   John|     1000|     null|     3000|
|          6|Michael|     null|     2000|     3000|
+-----------+-------+---------+---------+---------+



In [12]:
from pyspark.sql.functions import mean, when, col

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FillNullWithMean").getOrCreate()

mean_values = df.select([mean(col).alias(col) for col in df.columns if col != 'Customer_id' and col != 'Name'])

df_filled = df
for col in mean_values.columns:
    df_filled = df_filled.withColumn(col, when(df_filled[col].isNull(), mean_values.collect()[0][col]).otherwise(df_filled[col]))

df_filled.show()

+-----------+-------+---------+---------+---------+
|Customer_id|   Name|Product_1|Product_2|Product_3|
+-----------+-------+---------+---------+---------+
|          1|  Alice|   2000.0|   3000.0|   3000.0|
|          3|  Carol|   1500.0|   4000.0|   3000.0|
|          5|   John|   1000.0|   3000.0|   3000.0|
|          6|Michael|   1500.0|   2000.0|   3000.0|
+-----------+-------+---------+---------+---------+



In [15]:
from pyspark.sql.functions import col

df_filled = df_filled.withColumn('Total_Purchase', col('Product_1') + col('Product_2') + col('Product_3'))

df_sorted = df_filled.orderBy('Total_Purchase', ascending=False)

df_sorted.show()

+-----------+-------+---------+---------+---------+--------------+
|Customer_id|   Name|Product_1|Product_2|Product_3|Total_Purchase|
+-----------+-------+---------+---------+---------+--------------+
|          3|  Carol|   1500.0|   4000.0|   3000.0|        8500.0|
|          1|  Alice|   2000.0|   3000.0|   3000.0|        8000.0|
|          5|   John|   1000.0|   3000.0|   3000.0|        7000.0|
|          6|Michael|   1500.0|   2000.0|   3000.0|        6500.0|
+-----------+-------+---------+---------+---------+--------------+

